In [2]:
import cv2
import pyautogui
import time
import mss
import numpy as np
from matplotlib import pyplot as plt

pyautogui.FAILSAFE = True

---
# Work in progress

In [3]:
# Setup
threshold = 0.2  # Acceptable detection error
min_th, max_th = (1-threshold, 1+threshold)
img_path = 'mahjong.png'
pattern_path = 'pattern.png'

In [4]:
def get_ratios():
    """Helper function to understand a tile shape.
    Unfortunately, resizing game window can change tile shape,
    thus different approach have to be considered."""
    
    img_rgb = cv2.imread(img_path, 0)
    i_w, i_h = img_rgb.shape[::-1]
    template = cv2.imread(pattern_path, 0)
    t_w, t_h = template.shape[::-1]
    rec_ratio = t_w/t_h  # Expected tile ratio
    height_ratio = t_h/i_h  # Expected tile-to-board height ratio
    return rec_ratio, height_ratio, i_w, i_h

rec_ratio, height_ratio, i_w, i_h = get_ratios()

In [5]:
def grab_screen():
    """Capture a screen image."""
    try:
        img = np.array(sct.grab(monitor))
    except NameError:
        return "Initiate mss first."
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return img

In [6]:
def loc_tiles(screen):
    """Localize tiles within an image."""
    
    tiles = []
    img_gray = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
    binary = cv2.bitwise_not(img_gray)
    (contours,_) = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        if (w/h > (rec_ratio * min_th)
            and w/h < (rec_ratio * max_th)
            and h/i_h > (height_ratio * min_th)
            and h/i_h < (height_ratio * max_th)):
            tiles.append((x, y, h, w))
    print('%d tiles detected.' % len(tiles))
    return tiles

In [7]:
def clickable(tile):
    """Check whether tile is clickable."""
    x, y, w, h = tile

    img = grab_screen()
    pre_click = img[y:y+h, x:x+w]

    pyautogui.click(x=(x+w/2), y=(y+h/2))

    img = grab_screen()
    post_click = img[y:y+h, x:x+w]

    difference = cv2.subtract(pre_click, post_click)
    b, g, r = cv2.split(difference)
    if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
        return False
    else:
        return tile

Make `loc_siblings` look only among clickable tiles.

In [127]:
def loc_siblings(tile, clickables=[]):
    """Finds matching tiles."""
    x, y, h, w = tile
    screen = grab_screen()
    look_for = screen[y:y+h, x:x+w]
    result = cv2.matchTemplate(screen, look_for, cv2.TM_CCOEFF_NORMED)
    pattern_threshold = 0.8
    loc = np.where(result >= pattern_threshold) #  (array(y), array(x))
    sibling = zip(*loc[::-1])
    last_pressed = [0, 0]  # Way to limit double-taping tiles
    
    if loc[::-1][0].size == 1:
        # Found only 1 tile of this type.
        return (*list(sibling)[0],h,w)
        
    for pt in sibling:
        pt_x = pt[0] + w/2  # Center of tile
        pt_y = pt[1] + h/2  #

        if abs(last_pressed[0] - pt[0]) < 20 and abs(last_pressed[1] - pt[1]) < 20:
            #  Avoid clicking same tile twice.
            pass
        else:
            for ctile in clickables:
                cx, cy, ch, cw = ctile
                if (cx < pt_x < cx+cw
                    and cy < pt_y < cy+ch):
                    clickables.pop(clickables.index(ctile))
                    pyautogui.click(x=(pt[0]+w/2), y=(pt[1]+h/2))
                    #time.sleep(1)
                    last_pressed = pt

In [81]:
with mss.mss() as sct:
    # Part of the screen to capture
    monitor = {"top": 0, "left": 0, "width": 960, "height": 1080}

    while "Screen capturing":
        last_time = time.time()

        img = grab_screen()
        tiles = loc_tiles(img)
        clickable_tiles = []
        lonely = []
        
        for tile in tiles:
            clicked = clickable(tile)
            if clicked is not False:
                clickable_tiles.append(tile)
        
        img = grab_screen()
        check_tiles = loc_tiles(img)
        if len(check_tiles) == len(tiles):
            print('Found {} clickable tiles.'.format(len(clickable_tiles)))
            for tile in clickable_tiles:
                output = loc_siblings(tile, clickable_tiles)
        else:
            pass
        
        print("fps: {}".format(1 / (time.time() - last_time)))

        # Press "q" to quit
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            breakq

46 tiles detected.
45 tiles detected.
fps: 0.19384016536501075
45 tiles detected.
45 tiles detected.
Found 23 clickable tiles.
fps: 0.11836602494486177
44 tiles detected.
44 tiles detected.
Found 22 clickable tiles.
fps: 0.12264327213563772
43 tiles detected.
43 tiles detected.
Found 23 clickable tiles.
fps: 0.12130779529459938
40 tiles detected.
40 tiles detected.
Found 23 clickable tiles.
fps: 0.12464760035395689
38 tiles detected.
38 tiles detected.
Found 21 clickable tiles.
fps: 0.13709804159476677
35 tiles detected.
32 tiles detected.
fps: 0.25436986858569993
32 tiles detected.
32 tiles detected.
Found 19 clickable tiles.
fps: 0.15606757854032097
31 tiles detected.
31 tiles detected.
Found 19 clickable tiles.
fps: 0.15281779005348525
29 tiles detected.
29 tiles detected.
Found 17 clickable tiles.
fps: 0.16926270088500947
28 tiles detected.
26 tiles detected.
fps: 0.31657798118950575
26 tiles detected.
26 tiles detected.
Found 15 clickable tiles.
fps: 0.19479987969169418
23 tiles d

0 tiles detected.
Found 0 clickable tiles.
fps: 72.86331735112223
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 61.355217155980746
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 65.47972835844196
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 75.93287108278871
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 78.0234016035121
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 78.75739822742977
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 79.7608488951432
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 78.67171849795551
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 79.81852782218162
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 78.75000469386606
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 78.79290652239254
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 73.55976078149389
0 tiles det

0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 67.30485573991463
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 63.260595457150615
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 66.65242817187897
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 75.64664718825524
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 80.67210340052316
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 79.29040795493214
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 80.8540530120482
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 80.83068028521873
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 81.013346725127
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 81.70615966026415
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 82.30257839174287
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 81.585372495

0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 59.26083332155926
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 54.603379592261824
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 72.32057383267812
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 57.987640154290695
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 77.50007390983001
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 78.39380969291442
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 78.8017885995566
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 60.89468335317518
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 75.80798149219201
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 75.09272222719542
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 52.370537776723395
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 40.89131

0 tiles detected.
Found 0 clickable tiles.
fps: 63.42896893808789
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 62.96619227766769
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 63.6474605077467
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 69.70756190792754
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 71.53852976292
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 74.41195046659334
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 77.0487719749435
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 81.74597049250619
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 81.62188880456146
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 80.41226993865031
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 64.80091462472576
0 tiles detected.
0 tiles detected.
Found 0 clickable tiles.
fps: 70.52213535098781
0 tiles detecte

KeyboardInterrupt: 

---
# Work in progress

In [ ]:
class Mahjong:
    def __init__(self):
        threshold = 0.2
        self.min_th, self.max_th = (1-threshold, 1+threshold)
        self.img_path = '/home/jakub/Pictures/mahjong.png'
        self.pattern_path = '/home/jakub/Pictures/pattern.png'
        self.rec_ratio, self.height_ratio, self.i_w, self.i_h = self.get_ratios()
        
    def get_ratios(self):
        """Helper function to understand a tile shape.
        Unfortunately, resizing game window can change tile shape,
        thus different approach have to be considered."""
        
        img_rgb = cv2.imread(self.img_path, 0)
        i_w, i_h = img_rgb.shape[::-1]
        template = cv2.imread(self.pattern_path, 0)
        t_w, t_h = template.shape[::-1]
        rec_ratio = t_w/t_h  # Expected tile ratio
        height_ratio = t_h/i_h  # Expected tile-to-board height ratio
        return rec_ratio, height_ratio, i_w, i_h
    
    def grab_screen():
        """mss has to be initialized"""
        img = np.array(sct.grab(monitor))
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        return img
    
    def clickable(tile):
    """Check whether tile is clickable."""
    x, y, w, h = tile

    img = grab_screen()
    pre_click = img[y:y+h, x:x+w]

    pyautogui.click(x=(x+w/2), y=(y+h/2))

    img = grab_screen()
    post_click = img[y:y+h, x:x+w]

    difference = cv2.subtract(pre_click, post_click)
    b, g, r = cv2.split(difference)
    if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
        return False
    else:
        return tile
    
    def loc_tiles(self):
        self.tiles = []
        self.lonely = []
        self.screen = self.grab_screen().copy()
        img_gray = cv2.cvtColor(self.screen, cv2.COLOR_BGR2GRAY)
        binary = cv2.bitwise_not(img_gray)
        (contours,_) = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        for contour in contours:
            (x,y,w,h) = cv2.boundingRect(contour)
            if (w/h > (self.rec_ratio * self.min_th)
                and w/h < (self.rec_ratio * self.max_th)
                and h/self.i_h > (self.height_ratio * self.min_th)
                and h/self.i_h < (self.height_ratio * self.max_th)):
                self.tiles.append((x, y, h, w))
        print('%d tiles detected.' % len(self.tiles))
    
    def loc_siblings(self, tile, clickables=[]):
    """Finds matching tiles."""
    x, y, h, w = tile
    screen = self.grab_screen()
    look_for = screen[y:y+h, x:x+w]
    res = cv2.matchTemplate(screen, look_for, cv2.TM_CCOEFF_NORMED)
    pattern_threshold = 0.8
    loc = np.where(res >= pattern_threshold)
    look = zip(*loc[::-1])
    last_pressed = [0, 0]
    for pt in look:
        pt_x = pt[0] + w/2  # Center of tile
        pt_y = pt[1] + h/2  # Center of tile
        if loc[::-1][0].size == 1:
            # Detect unique tiles (need it to detect
            # gold and black tiles).
            return pt+(h,w)
        
        if abs(last_pressed[0] - pt[0]) < 20 and abs(last_pressed[1] - pt[1]) < 20:
            pass
        else:
            for ctile in clickables:
                cx, cy, ch, cw = ctile
                if (cx < pt_x < cx+cw
                    and cy < pt_y < cy+ch):
                    pyautogui.click(x=(pt[0]+w/2), y=(pt[1]+h/2))
                    #time.sleep(1)
                    last_pressed = pt

    def solve(self):
        with mss.mss() as sct:
            # Part of the screen to capture
            monitor = {"top": 0, "left": 0, "width": 960, "height": 1080}
        
        start = time.time()
        self.loc_tiles()
        for tile in range(len(self.tiles)):
            self.loc_siblings(tile)
        print(self.lonely)
        stop = time.time()
        return stop - start